In [1]:
!ls data/finance_sales

2003_bronx.xls        2007_statenisland.xls 2012_queens.xls
2003_brooklyn.xls     2008_bronx.xls        2012_statenisland.xls
2003_manhattan.xls    2008_brooklyn.xls     2013_bronx.xls
2003_queens.xls       2008_manhattan.xls    2013_brooklyn.xls
2003_statenisland.xls 2008_queens.xls       2013_manhattan.xls
2004_bronx.xls        2008_statenisland.xls 2013_queens.xls
2004_brooklyn.xls     2009_bronx.xls        2013_statenisland.xls
2004_manhattan.xls    2009_brooklyn.xls     2014_bronx.xls
2004_queens.xls       2009_manhattan.xls    2014_brooklyn.xls
2004_statenisland.xls 2009_queens.xls       2014_manhattan.xls
2005_bronx.xls        2009_statenisland.xls 2014_queens.xls
2005_brooklyn.xls     2010_bronx.xls        2014_statenisland.xls
2005_manhattan.xls    2010_brooklyn.xls     2015_bronx.xls
2005_queens.xls       2010_manhattan.xls    2015_brooklyn.xls
2005_statenisland.xls 2010_queens.xls       2015_manhattan.xls
2006_bronx.xls        2010_statenisland.xls 2015_queens.xls
2006_brook

In [2]:
import platform
print(platform.python_version())

3.5.2


In [3]:
import pandas as pd
import numpy as np

In [4]:
from merge_pluto_finance import *

In [5]:
boros = ['brooklyn', 'manhattan']
years = [2014, 2015]

In [6]:
pluto = read_in_pluto(boros)

In [7]:
finance = read_in_finance(boros, years)

Pulling Finance data for 2014_brooklyn
Pulling Finance data for 2014_manhattan
Pulling Finance data for 2015_brooklyn
Pulling Finance data for 2015_manhattan


In [12]:
finance.bbl.value_counts()[finance.bbl.value_counts() > 1]

1010061302    1712
1010090037    1481
1010061303    1169
1013731901     225
3000860011      96
1019780001      71
1011581007      70
1007510001      69
3019020001      67
1021800633      66
1013280021      65
1003110013      63
1003210001      63
1013380005      62
1002630008      60
1003150001      60
1013340041      59
1013340022      55
1003310120      55
1021790010      51
1005730043      49
1011840031      49
1009170017      48
1013330023      48
1011380001      48
1013670001      47
1018741002      47
1010090029      47
3059060018      47
1007480001      46
              ... 
3018550054       2
3007240064       2
3036920105       2
3018930047       2
3046750058       2
3027320010       2
1019020033       2
3054960004       2
3002510026       2
1006380004       2
1011141531       2
3074551201       2
1011630008       2
3034410024       2
3040710035       2
1011990027       2
1008980039       2
3052250070       2
3051590061       2
3015520059       2
3041540089       2
1007060017  

In [ ]:
#finance = finance[finance["sale price"] > 0]

In [10]:
finance[finance.bbl]

,borough,neighborhood,building class category,tax class at present,block,lot,ease-ment,building class at present,address,apartment number,...,total units,land square feet,gross square feet,year built,tax class at time of sale,building class at time of sale,sale price,sale date,bbl,price per sqft
10867,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,28000,2014-08-24,1010090037,0.248117
10868,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,170810,2014-07-23,1010090037,1.513602
10869,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,58900,2014-07-23,1010090037,0.521932
10870,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,114855,2014-07-23,1010090037,1.017767
10871,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,139000,2014-07-23,1010090037,1.231724
10872,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,58900,2014-07-23,1010090037,0.521932
10873,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,81305,2014-07-23,1010090037,0.720470
10874,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,58900,2014-07-23,1010090037,0.521932
10875,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,42400,2014-07-23,1010090037,0.375720
10876,1,MIDTOWN WEST,25 LUXURY HOTELS,4,1009,37,,H3,102 WEST 57TH STREET,,...,2,7532,112850,2007,4,H2,58900,2014-07-23,1010090037,0.521932
